In [1]:
import os
import numpy as np

import polynomials

In [2]:
ring = polynomials.PolynomialRing(sofile=os.path.join(".", 'polynomialslib.so'))

polynomial1 = ring.polynomial_from_terms([(1.0,[1,2,0]), (1.0,[0,0,0])])
polynomial2 = ring.polynomial_from_terms([(1.0,[1,1,0]), (1.0,[0,0,0])])
polynomial3 = ring.polynomial_from_terms([(1.0,[0,1,0]), (1.0,[0,0,0])])

quotients, remainder = ring.div(polynomial1, [polynomial2, polynomial3])
assert(ring.add(ring.mul(quotients[0], polynomial2), ring.mul(quotients[1], polynomial3), remainder).terms() == polynomial1.terms())

In [3]:
polynomial1 = ring.polynomial_from_terms([(1.0,[2,0,0]), (-2.0,[1,0,1]), (5.0,[0,0,0])])
polynomial2 = ring.polynomial_from_terms([(1.0,[1,2,0]), (1.0,[0,1,1]), (1.0,[0,0,0])])
polynomial3 = ring.polynomial_from_terms([(3.0,[0,2,0]), (-8.0,[1,0,1])])

groebner = ring.buchbergers(True, polynomial1, polynomial2, polynomial3)
quotients, remainder = ring.div(ring.polynomial_from_terms([(1.0,[2,0,0])]), groebner)
print quotients, remainder

[[(1.0, [2, 0, 0])]] []


In [4]:
polynomial1 = ring.polynomial_from_terms([(1.0,[2,0,0]), (-2.0,[1,0,1]), (5.0,[0,0,0])])
polynomial2 = ring.polynomial_from_terms([(1.0,[1,2,0]), (1.0,[0,1,1]), (1.0,[0,0,0])])
polynomial3 = ring.polynomial_from_terms([(3.0,[0,2,0]), (-8.0,[1,0,1])])

groebner = ring.buchbergers(False, polynomial1, polynomial2, polynomial3)
print polynomials.find_standard_monomial_basis(groebner)

set([(0, 1, 2), (1, 1, 0), (0, 0, 2), (0, 0, 3), (0, 0, 4)])


### Dense (Spectral) Multiplication

In [5]:
# Max Degree = 3
class DensePolynomials3(object):
    @staticmethod
    def make(x3=0, x2y=0, x2z=0, xy2=0, xyz=0, xz2=0, y3=0, y2z=0, yz2=0,
             z3=0, x2=0, xy=0, xz=0, y2=0, yz=0, z2=0, x=0, y=0, z=0, c=0):
        polynomial = np.zeros((4,4,4))
        polynomial[3,0,0] = x3
        polynomial[2,1,0] = x2y
        polynomial[2,0,1] = x2z
        polynomial[1,2,0] = xy2
        polynomial[1,1,1] = xyz
        polynomial[1,0,2] = xz2
        polynomial[0,3,0] = y3
        polynomial[0,2,1] = y2z
        polynomial[0,1,2] = yz2
        polynomial[0,0,3] = z3
        polynomial[2,0,0] = x2
        polynomial[1,1,0] = xy
        polynomial[1,0,1] = xz
        polynomial[0,2,0] = y2
        polynomial[0,1,1] = yz
        polynomial[0,0,2] = z2
        polynomial[1,0,0] = x
        polynomial[0,1,0] = y
        polynomial[0,0,1] = z
        polynomial[0,0,0] = c
        return polynomial

    @staticmethod
    def grlex_coeffs(polynomial):
        return np.array([polynomial[3,0,0], polynomial[2,1,0], polynomial[2,0,1], polynomial[1,2,0], 
                         polynomial[1,1,1], polynomial[1,0,2], polynomial[0,3,0], polynomial[0,2,1], 
                         polynomial[0,1,2], polynomial[0,0,3], polynomial[2,0,0], polynomial[1,1,0], 
                         polynomial[1,0,1], polynomial[0,2,0], polynomial[0,1,1], polynomial[0,0,2], 
                         polynomial[1,0,0], polynomial[0,1,0], polynomial[0,0,1], polynomial[0,0,0]])
    
    @staticmethod
    def grevlex_coeffs(polynomial):
        return np.array([polynomial[3,0,0], polynomial[2,1,0], polynomial[1,2,0], polynomial[0,3,0], 
                         polynomial[2,0,1], polynomial[1,1,1], polynomial[0,2,1], polynomial[1,0,2], 
                         polynomial[0,1,2], polynomial[0,0,3], polynomial[2,0,0], polynomial[1,1,0], 
                         polynomial[0,2,0], polynomial[1,0,1], polynomial[0,1,1], polynomial[0,0,2], 
                         polynomial[1,0,0], polynomial[0,1,0], polynomial[0,0,1], polynomial[0,0,0]])
    
    @staticmethod
    def mul(polynomial_left, polynomial_right):
        return np.real(np.fft.ifftn(np.fft.fftn(polynomial_left)*np.fft.fftn(polynomial_right))).astype(np.float64)